<img src="https://dutchanalytics.com/wp-content/uploads/2020/02/svg_logo_dutch_analytics-1.svg" width="80">

# Quickstart
Deploy a simple float-to-float deployment in UbiOps using the python client library.


The first step is to download a prepared deployment package (the deployment code).

In [ ]:
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/example_deployment_package.zip -o simple_deployment_package.zip

Add your API token, provide a project name, deployment name and version name. 
Afterward initialize the client library.

In [ ]:
API_TOKEN = 'Token <YOUR_API_KEY>'
PROJECT_NAME = '<YOUR_PROJECT_NAME>'
DEPLOYMENT_NAME = 'simple-tutorial'
DEPLOYMENT_VERSION = 'v1'

In [ ]:
import ubiops
configuration = ubiops.Configuration()
configuration.api_key['Authorization'] = API_TOKEN

Here we open the connection with the UbiOps API Client.

In [ ]:
client = ubiops.ApiClient(configuration)
api = ubiops.CoreApi(client)
api.service_status()

## Deploy
Create a deployment.

In [ ]:
deployment_template = ubiops.DeploymentCreate(
    name=DEPLOYMENT_NAME,
    description='A simple deployment that multiplies the input float by a random number.',
    input_type='structured',
    output_type='structured',
    input_fields=[ubiops.DeploymentInputFieldCreate(name='input', data_type='double')],
    output_fields=[ubiops.DeploymentOutputFieldCreate(name='output', data_type='double')]
)

deployment = api.deployments_create(project_name=PROJECT_NAME, data=deployment_template)
print(deployment)

Create a deployment version.

In [ ]:
version_template = ubiops.VersionCreate(
    version=DEPLOYMENT_VERSION,
    language='python3.7',
    memory_allocation=256,
    maximum_instances=1,
    minimum_instances=0,
    maximum_idle_time=1800 # = 30 minutes
)

version = api.versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=version_template
)
print(version)

Upload the prepared deployment file to the created deployment version.

In [ ]:
file_upload_result =api.versions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    file='simple_deployment_package.zip'
)

Check if deployment version is finished building.
This can take a few minutes.

In [ ]:
from time import sleep
status = 'building'
while status != 'available' and 'failed' not in status:    
    version_status = api.versions_get(       
        project_name=PROJECT_NAME,        
        deployment_name=DEPLOYMENT_NAME,        
        version=DEPLOYMENT_VERSION    
    )    
    status = version_status.status
    print(status, end='\r')
    sleep(1)
print(status)

## Creating a direct request
Make sure the deployment is in 'available' state before performing deployment requests.

The first request is usually slow because of a cold start. A second request (performed within the `maximum_idle_time`, which was set to 30 minutes in this tutorial) will be much faster.

In [ ]:
data = {'input': 123}
request_result = api.deployment_requests_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    data=data
)
print(request_result)

***

# Pipelines

Multiple deployments can be chained using a *pipeline*. In this tutorial, we will chain the deployment created above with a new deployment. Just for demo purposes, we will use the same deployment package (the same code) for both deployments in the pipeline. Therefore, the pipeline will contain two modelling steps: each step will multiply the input by a random number.

## Create another deployment and version
Normally you would like to chain different deployments. However, just for demo purposes, we will use the same deployment package as used above.

Create the deployment and version

In [ ]:
DEPLOYMENT_NAME2 = f"{DEPLOYMENT_NAME}-2"
deployment_template2 = deployment_template
deployment_template2.name = DEPLOYMENT_NAME2

api.deployments_create(
    project_name=PROJECT_NAME,
    data=deployment_template2
)

api.versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME2,
    data=version_template
)

Upload the zip

In [ ]:
api.versions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME2,
    version=DEPLOYMENT_VERSION,
    file='simple_deployment_package.zip'
)

Wait for the deployment to be ready

In [ ]:
from time import sleep
status = 'building'
print(status, end='\r')
while status != 'available' and 'failed' not in status:    
    version_status = api.versions_get(       
        project_name=PROJECT_NAME,        
        deployment_name=DEPLOYMENT_NAME2,        
        version=DEPLOYMENT_VERSION    
    )    
    status = version_status.status
    print(status, end='\r')
    sleep(1)
print(status)

## Create a pipeline

In [ ]:
PIPELINE_NAME = f"{DEPLOYMENT_NAME}-pipeline"

In [ ]:
pipeline_template = ubiops.PipelineCreate(
    name=PIPELINE_NAME,
    description='A simple pipeline that multiplies the input float two times by a random number.',
    input_type='structured',
    input_fields=[ubiops.DeploymentInputFieldCreate(name='input', data_type='double')]
)

api.pipelines_create(project_name=PROJECT_NAME, data=pipeline_template)

## Add deployments as objects to pipeline

In [ ]:
object_template = ubiops.PipelineObjectCreate(
    name=DEPLOYMENT_NAME,
    reference_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION
)
api.pipeline_objects_create(project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, data=object_template)

In [ ]:
object_template2 = ubiops.PipelineObjectCreate(
    name=DEPLOYMENT_NAME2,
    reference_name=DEPLOYMENT_NAME2,
    version=DEPLOYMENT_VERSION
)
api.pipeline_objects_create(project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, data=object_template2)

## Connect the objects

start -> deployment1

In [ ]:
connection_template = ubiops.AttachmentsCreate(
    source_name='pipeline_start', 
    destination_name=DEPLOYMENT_NAME,
    mapping=[ubiops.AttachmentFieldsCreate(source_field_name='input', destination_field_name='input')]
)

api.pipeline_object_attachments_create(
    project_name=PROJECT_NAME, 
    pipeline_name=PIPELINE_NAME, 
    data=connection_template
)

deployment1 -> deployment2

In [ ]:
connection_template = ubiops.AttachmentsCreate(
    source_name=DEPLOYMENT_NAME, 
    destination_name=DEPLOYMENT_NAME2,
    mapping=[ubiops.AttachmentFieldsCreate(source_field_name='output', destination_field_name='input')]
)

api.pipeline_object_attachments_create(
    project_name=PROJECT_NAME, 
    pipeline_name=PIPELINE_NAME, 
    data=connection_template
)

## Creating a direct request

In [ ]:
data = {'input': 123}
pipeline_result = api.pipeline_requests_create(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    data=data
)
print(pipeline_result)

## Cleanup
Delete created deployment.

In [ ]:
api.pipelines_delete(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME
)
api.deployments_delete(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME
)
api.deployments_delete(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME2
)

## Close connection
Close the connection with the UbiOps API client.

In [ ]:
client.close()